In [5]:
import cv2
import mediapipe as mp
import time
import threading
from plyer import notification

In [ ]:
MIN_BLINK_RATE = 10    # Минимальное количество морганий в минуту
MAX_FOCUS_TIME = 20    # Максимальное время фокусировки на одном объекте
MAX_WORK_TIME = 60 * 20  # Максимальное время работы без перерыва

blink_count = 0
last_blink_time = time.time()
focus_start_time = time.time()
work_start_time = time.time()

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

def calculate_eye_aspect_ratio(eye_points):
    """Вычисление EAR (Eye Aspect Ratio)"""
    p1, p2, p3, p4, p5, p6 = eye_points
    ear = (abs(p2[1] - p6[1]) + abs(p3[1] - p5[1])) / (2 * abs(p1[0] - p4[0]))
    return ear

def detect_blinks(face_landmarks, frame):
    """Обнаружение морганий"""
    global blink_count, last_blink_time

    left_eye_indices = [33, 160, 158, 133, 153, 144]
    right_eye_indices = [263, 387, 385, 362, 380, 373]

    height, width, _ = frame.shape
    left_eye = [(int(face_landmarks.landmark[i].x * width), int(face_landmarks.landmark[i].y * height)) for i in left_eye_indices]
    right_eye = [(int(face_landmarks.landmark[i].x * width), int(face_landmarks.landmark[i].y * height)) for i in right_eye_indices]

    left_ear = calculate_eye_aspect_ratio(left_eye)
    right_ear = calculate_eye_aspect_ratio(right_eye)

    if left_ear < 0.2 and right_ear < 0.2:
        current_time = time.time()
        if current_time - last_blink_time > 0.5:  
            blink_count += 1
            last_blink_time = current_time

def check_focus_time():
    """Проверка времени фокусировки"""
    global focus_start_time

    current_time = time.time()
    if current_time - focus_start_time > MAX_FOCUS_TIME:
        notification.notify(
            title="Внимание!",
            message="Вы долго фокусируетесь на одном объекте. Переведите взгляд на дальний объект.",
            timeout=5
        )
        focus_start_time = current_time

def check_work_time():
    """Проверка времени работы без перерыва."""
    global work_start_time

    current_time = time.time()
    if current_time - work_start_time > MAX_WORK_TIME:
        notification.notify(
            title="Время отдохнуть!",
            message="Вы долго работаете без перерыва. Сделайте паузу.",
            timeout=10
        )
        work_start_time = current_time

def eye_tracking():
    """Основной цикл отслеживания глаз."""
    global blink_count

    cap = cv2.VideoCapture(0) 
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                detect_blinks(face_landmarks, frame)
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )

        check_focus_time()
        check_work_time()

        cv2.putText(frame, f"Blinks: {blink_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Eye Tracking", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    tracking_thread = threading.Thread(target=eye_tracking)
    tracking_thread.daemon = True
    tracking_thread.start()

    while True:
        time.sleep(1)

if __name__ == "__main__":
    main()

KeyboardInterrupt: 